# CWRU Bearing Fault Data - Data Feature Extraction (divided by load)

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import os
import scipy.io
import pickle
import librosa

from scipy.stats import skew, kurtosis, entropy
from scipy.fft import fft
from scipy.signal import stft
from IPython.display import display_html

In [2]:
def extract_time_domain_features(signal):
    signal = np.array(signal)

    zcr = librosa.feature.zero_crossing_rate(signal, frame_length=len(signal), hop_length=len(signal))[0][0]
    rms = np.sqrt(np.mean(signal**2))
    mean = np.mean(signal)
    avg = np.mean(np.abs(signal))

    return {
        'mean': mean,
        'std': np.std(signal),
        'variance': np.var(signal),
        'min': np.min(signal),
        'max': np.max(signal),
        'peak_to_peak': np.ptp(signal),
        'skewness': skew(signal),
        'kurtosis': kurtosis(signal),
        'rms': np.sqrt(np.mean(signal**2)), # root mean square
        'zcr': zcr, # zero crossing rate
        'crest_factor': np.max(np.abs(signal)) / rms,
        'form_factor': rms / avg
    }

In [3]:
def extract_frequency_domain_features(fft_amplitudes):
    fft_amplitudes = np.array(fft_amplitudes)
    freqs = np.linspace(0, SAMPLE_RATE / 2, len(fft_amplitudes))

    norm_fft_amplitudes = fft_amplitudes / (np.sum(fft_amplitudes) + 1e-12)
    power_spectrum = fft_amplitudes**2

    peak_idx = np.argmax(fft_amplitudes)
    peak_freq = freqs[peak_idx]

    cumulative_power = np.cumsum(power_spectrum)
    median_freq = freqs[np.searchsorted(cumulative_power, cumulative_power[-1] / 2)]

    low_band = power_spectrum[freqs < 3000]
    high_band = power_spectrum[freqs >= 3000]
    band_energy_ratio = np.sum(high_band) / (np.sum(low_band) + 1e-12)

    return {
        'mean': np.mean(fft_amplitudes), # mean magnitude
        'std': np.std(fft_amplitudes), # standard deviation magnitude
        'max': np.max(fft_amplitudes), # max magnitude
        'skewness': skew(fft_amplitudes), # skewness magnitude
        'kurtosis': kurtosis(fft_amplitudes), # kurtosis magnitude
        'spec_centroid': np.sum(freqs * norm_fft_amplitudes), # spectral centroid
        'spec_spread': np.sqrt(np.sum(((freqs - np.sum(freqs * norm_fft_amplitudes))**2) * norm_fft_amplitudes)), # spectral_spread
        'spec_flatness': np.exp(np.mean(np.log(fft_amplitudes + 1e-12))) / (np.mean(fft_amplitudes) + 1e-12), # spectral_flatness
        'spec_entropy': entropy(norm_fft_amplitudes), # spectral_entropy
        'total_power': np.sum(power_spectrum),
        'dominant_freq': peak_freq, # dominant frequency
        'median_freq': median_freq, # median frequency
        'ber': band_energy_ratio, # band energy ratio
    }

In [4]:
def extract_time_frequency_domain_features(magnitude, freqs):
    mean_mag = np.mean(magnitude)
    max_mag = np.max(magnitude)
    spec_centroid = np.sum(freqs[:, None] * magnitude) / (np.sum(magnitude) + 1e-12)
    spec_spread = np.sqrt(np.sum(((freqs[:, None] - spec_centroid)**2) * magnitude) / (np.sum(magnitude) + 1e-12))
    total_power = np.sum(magnitude**2)

    return {
        'mean': mean_mag, # mean magnitude
        'max': max_mag, # max_magnitude
        'spec_centroid': spec_centroid, # spectral centroid
        'spec_spread': spec_spread, # spectral spread
        'total_power': total_power
    }

## 48k Drive-End Bearing Fault Data + Normal Baseline Data (48k)

In [5]:
with open('48kdrive-end_normalbaseline_loadsplit_cropped_data.pkl', 'rb') as f:
    data = pickle.load(f)

In [6]:
SAMPLE_RATE = 48000
FRAME_SIZE = 1024
HOP_SIZE = 512

### Feature extraction in time domain

In [7]:
load_feature_tables = {}

for load in data['load'].unique():
    load_df = data[data['load'] == load]

    feature_rows = []
    for _, row in load_df.iterrows():
        features = extract_time_domain_features(row['cropped_signal'])
        features['label'] = row['label']
        feature_rows.append(features)

    features_df = pd.DataFrame(feature_rows)
    load_feature_tables[load] = features_df

#### Mean, Standard deviation, Variance

In [8]:
tables = []

for load in sorted(load_feature_tables.keys()):
    df = load_feature_tables[load][['label', 'mean', 'variance', 'std']].copy()
    html_table = df.to_html(index=False)
    tables.append(f'<div style="display:inline-block; margin-right:30px;">{html_table}</div>')

display_html(''.join(tables), raw=True)

label,mean,variance,std
Normal,0.012059,0.005229,0.072313
B007,0.032624,0.022016,0.148378
B014,0.033902,0.012497,0.111790
B021,0.285251,0.116771,0.341718
OR021@12,0.116960,0.387444,0.622450
OR007@12,0.119367,0.046886,0.216533
OR021@3,0.006357,0.171236,0.413807
OR007@3,0.109014,0.162601,0.403238
IR007,0.082329,0.341184,0.584110
IR014,0.037581,0.031744,0.178169


#### Min, Max, Peak to Peak

In [9]:
tables = []

for load in sorted(load_feature_tables.keys()):
    df = load_feature_tables[load][['label', 'min', 'max', 'peak_to_peak']].copy()
    html_table = df.to_html(index=False)
    tables.append(f'<div style="display:inline-block; margin-right:8px;">{html_table}</div>')

display_html(''.join(tables), raw=True)

label,min,max,peak_to_peak
Normal,-0.286638,0.311254,0.597892
B007,-0.523833,0.679252,1.203085
B014,-0.711378,0.811097,1.522475
B021,-1.125131,1.575851,2.700981
OR021@12,-5.212493,5.788413,11.000907
OR007@12,-1.279021,1.471156,2.750177
OR021@3,-2.971831,2.479377,5.451208
OR007@3,-1.850210,1.992903,3.843113
IR007,-2.772763,3.291508,6.064271
IR014,-1.319191,1.449816,2.769007


#### Skewness, Kurtosis

In [10]:
tables = []

for load in sorted(load_feature_tables.keys()):
    df = load_feature_tables[load][['label', 'skewness', 'kurtosis']].copy()
    html_table = df.to_html(index=False)
    tables.append(f'<div style="display:inline-block; margin-right:30px;">{html_table}</div>')

display_html(''.join(tables), raw=True)

label,skewness,kurtosis
Normal,-0.045550,-0.185236
B007,-0.019090,0.105033
B014,0.002050,2.448639
B021,0.024113,0.084684
OR021@12,0.132761,21.360128
OR007@12,0.034581,2.555944
OR021@3,-0.262731,5.134962
OR007@3,0.072165,1.196439
IR007,0.005247,3.020826
IR014,-0.091302,8.669292


#### RMS (root mean square)

In [11]:
tables = []

for load in sorted(load_feature_tables.keys()):
    df = load_feature_tables[load][['label', 'rms']].copy()
    html_table = df.to_html(index=False)
    tables.append(f'<div style="display:inline-block; margin-right:100px;">{html_table}</div>')

display_html(''.join(tables), raw=True)

label,rms
Normal,0.073311
B007,0.151923
B014,0.116818
B021,0.445129
OR021@12,0.633343
OR007@12,0.247254
OR021@3,0.413856
OR007@3,0.417714
IR007,0.589883
IR014,0.182089


#### Zero crossing rate

In [12]:
tables = []

for load in sorted(load_feature_tables.keys()):
    df = load_feature_tables[load][['label', 'zcr']].copy()
    html_table = df.to_html(index=False)
    tables.append(f'<div style="display:inline-block; margin-right:100px;">{html_table}</div>')

display_html(''.join(tables), raw=True)

label,zcr
Normal,0.082915
B007,0.060199
B014,0.049523
B021,0.038377
OR021@12,0.039976
OR007@12,0.046310
OR021@3,0.055590
OR007@3,0.058898
IR007,0.063492
IR014,0.052690


### Feature extraction in frequency domain

In [13]:
load_fft_tables = {}
load_fft_feature_tables = {}

for load in data['load'].unique():
    load_df = data[data['load'] == load]

    fft_rows = []
    feature_rows = []

    for _, row in load_df.iterrows():
        signal = np.array(row['cropped_signal'])

        fft_result = np.fft.rfft(signal)
        fft_amplitudes = np.abs(fft_result)

        fft_dict = {f'fft_{i}': amp for i, amp in enumerate(fft_amplitudes)}
        fft_dict['label'] = row['label']
        fft_rows.append(fft_dict)

        features = extract_frequency_domain_features(fft_amplitudes)
        features['label'] = row['label']
        feature_rows.append(features)

    fft_df = pd.DataFrame(fft_rows)
    features_df = pd.DataFrame(feature_rows)

    load_fft_tables[load] = fft_df
    load_fft_feature_tables[load] = features_df

#### Mean magnitude, Standard deviation magnitude, Max magnitude

In [24]:
tables = []

for load in sorted(load_fft_feature_tables.keys()):
    df = load_fft_feature_tables[load][['label', 'mean', 'std', 'max']].copy()
    html_table = df.to_html(index=False)
    tables.append(f'<div style="display:inline-block; margin-right:0px;">{html_table}</div>')

display_html(''.join(tables), raw=True)

label,mean,std,max
Normal,4.354040,18.252052,1423.231877
B007,7.965570,38.427174,2081.009391
B014,7.575948,29.771998,2162.571122
B021,15.769366,132.589667,18195.609385
OR021@12,24.701168,160.774718,7460.632944
OR007@12,11.425461,68.394783,7614.152162
OR021@3,29.162845,100.384979,2346.956900
OR007@3,19.418944,107.287747,6953.798103
IR007,35.174226,146.253975,7247.623156
IR014,12.623060,45.229060,2397.219424


#### Skewness magnitude, Kurtosis magnitude

In [15]:
tables = []

for load in sorted(load_fft_feature_tables.keys()):
    df = load_fft_feature_tables[load][['label', 'skewness', 'kurtosis']].copy()
    html_table = df.to_html(index=False)
    tables.append(f'<div style="display:inline-block; margin-right:30px;">{html_table}</div>')

display_html(''.join(tables), raw=True)

label,skewness,kurtosis
Normal,36.318393,2076.028017
B007,15.161559,437.351517
B014,18.114617,940.427551
B021,86.283393,11191.845879
OR021@12,19.225282,526.509744
OR007@12,51.465482,4971.926219
OR021@3,7.536522,80.384802
OR007@3,21.366264,834.258361
IR007,19.275290,604.475032
IR014,10.254882,285.240881


#### Spectral centroid, Spectral spread

In [16]:
tables = []

for load in sorted(load_fft_feature_tables.keys()):
    df = load_fft_feature_tables[load][['label', 'spec_centroid', 'spec_spread']].copy()
    html_table = df.to_html(index=False)
    tables.append(f'<div style="display:inline-block; margin-right:8px;">{html_table}</div>')

display_html(''.join(tables), raw=True)

label,spec_centroid,spec_spread
Normal,4655.864957,4459.586913
B007,3175.777514,2477.661405
B014,3275.418309,2989.823842
B021,3122.831877,2954.562569
OR021@12,2894.816735,1926.907514
OR007@12,3884.589750,3873.537270
OR021@3,3198.848564,2616.324033
OR007@3,3522.761692,2596.180313
IR007,3770.064757,2874.897618
IR014,3200.120111,2625.505676


#### Spectral flatness, Spectral entropy

In [17]:
tables = []

for load in sorted(load_fft_feature_tables.keys()):
    df = load_fft_feature_tables[load][['label', 'spec_flatness', 'spec_entropy']].copy()
    html_table = df.to_html(index=False)
    tables.append(f'<div style="display:inline-block; margin-right:8px;">{html_table}</div>')

display_html(''.join(tables), raw=True)

label,spec_flatness,spec_entropy
Normal,0.253348,9.008821
B007,0.088189,8.128475
B014,0.119786,8.460957
B021,0.110667,7.978351
OR021@12,0.057750,7.743569
OR007@12,0.145476,8.317296
OR021@3,0.105807,8.530812
OR007@3,0.097051,8.129905
IR007,0.124383,8.616732
IR014,0.105035,8.495823


#### Total power, Median frequency

In [18]:
tables = []

for load in sorted(load_fft_feature_tables.keys()):
    df = load_fft_feature_tables[load][['label', 'total_power', 'median_freq']].copy()
    html_table = df.to_html(index=False)
    tables.append(f'<div style="display:inline-block; margin-right:8px;">{html_table}</div>')

display_html(''.join(tables), raw=True)

label,total_power,median_freq
Normal,1.123007e+07,4144.729416
B007,4.912143e+07,2907.631529
B014,3.010144e+07,2668.338873
B021,5.686462e+08,0.000000
OR021@12,8.438989e+08,2748.855584
OR007@12,1.533635e+08,2584.059698
OR021@3,3.485363e+08,2833.134759
OR007@3,3.791600e+08,2814.322443
IR007,7.217026e+08,2760.142974
IR014,7.032878e+07,2699.191070


#### Dominant frequency, Band energy ratio

In [19]:
tables = []

for load in sorted(load_fft_feature_tables.keys()):
    df = load_fft_feature_tables[load][['label', 'dominant_freq', 'ber']].copy()
    html_table = df.to_html(index=False)
    tables.append(f'<div style="display:inline-block; margin-right:30px;">{html_table}</div>')

display_html(''.join(tables), raw=True)

label,dominant_freq,ber
Normal,4144.729416,2.086419
B007,0.000000,0.739567
B014,0.000000,0.294075
B021,0.000000,0.037220
OR021@12,0.000000,0.040083
OR007@12,0.000000,0.163281
OR021@3,2748.855584,0.527710
OR007@3,0.000000,0.353216
IR007,2616.416881,0.515407
IR014,0.000000,0.229035


### Feature extraction in time-frequency domain

In [20]:
load_stft_tables = {}
load_stft_feature_tables = {}

for load in sorted(data['load'].unique()):
    load_df = data[data['load'] == load]

    stft_rows = []
    feature_rows = []

    for _, row in load_df.iterrows():
        signal = np.array(row['cropped_signal'])

        S = librosa.stft(signal, n_fft=FRAME_SIZE, hop_length=HOP_SIZE, win_length=FRAME_SIZE, window='hann', center=True)
        magnitude = np.abs(S)

        freqs = np.linspace(0, SAMPLE_RATE / 2, magnitude.shape[0])

        stft_dict = {f'stft_{i}': mag for i, mag in enumerate(magnitude.flatten())}
        stft_dict['label'] = row['label']
        stft_rows.append(stft_dict)

        features = extract_time_frequency_domain_features(magnitude, freqs)
        features['label'] = row['label']
        feature_rows.append(features)

    stft_df = pd.DataFrame(stft_rows)
    features_df = pd.DataFrame(feature_rows)

    load_stft_tables[load] = stft_df
    load_stft_feature_tables[load] = features_df


#### Mean magnitude for all frames, Max magnitude for all frames

In [21]:
tables = []

for load in sorted(load_stft_feature_tables.keys()):
    df = load_stft_feature_tables[load][['label', 'mean', 'max']].copy()
    html_table = df.to_html(index=False)
    tables.append(f'<div style="display:inline-block; margin-right:30px;">{html_table}</div>')

display_html(''.join(tables), raw=True)


label,mean,max
Normal,0.533445,16.965659
B007,0.887999,46.812279
B014,0.750646,42.458211
B021,2.185782,156.320575
OR021@12,2.187100,411.102242
OR007@12,1.384778,85.057663
OR021@3,2.566951,117.649251
OR007@3,2.415092,113.994387
IR007,4.669546,100.375157
IR014,0.988049,73.415319


#### Spectral centroid for all frames, Spectral spread for all frames

In [22]:
tables = []

for load in sorted(load_stft_feature_tables.keys()):
    df = load_stft_feature_tables[load][['label', 'spec_centroid', 'spec_spread']].copy()
    html_table = df.to_html(index=False)
    tables.append(f'<div style="display:inline-block; margin-right:30px;">{html_table}</div>')

display_html(''.join(tables), raw=True)


label,spec_centroid,spec_spread
Normal,4219.881204,4016.604835
B007,2900.507124,2418.854747
B014,2784.665734,2923.003153
B021,2445.110289,2630.757215
OR021@12,2594.939433,2015.218044
OR007@12,3195.984558,3479.159724
OR021@3,2943.755689,2426.532599
OR007@3,3116.148235,2381.608706
IR007,3314.499866,2540.556489
IR014,3043.482170,2705.370081


#### Total power for all frames

In [23]:
tables = []

for load in sorted(load_stft_feature_tables.keys()):
    df = load_stft_feature_tables[load][['label', 'total_power']].copy()
    html_table = df.to_html(index=False)
    tables.append(f'<div style="display:inline-block; margin-right:30px;">{html_table}</div>')

display_html(''.join(tables), raw=True)


label,total_power
Normal,1.338636e+05
B007,5.803393e+05
B014,3.495452e+05
B021,6.209870e+06
OR021@12,1.018344e+07
OR007@12,1.724693e+06
OR021@3,4.185402e+06
OR007@3,4.468484e+06
IR007,8.615453e+06
IR014,8.054196e+05
